In [11]:
import json

import torch
from PIL import Image
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import NeptuneLogger
from torch import nn
from torchvision import transforms as T

from config import FT_NET_CFG, SHAPE_EMBEDDING_CFG
from src.models.baseline import LitModule

In [2]:
edge_index = SHAPE_EMBEDDING_CFG.EDGE_INDEX

In [3]:
edge_index = torch.LongTensor(edge_index)

In [4]:
with open("./data/market1501/jsons/train.json") as f:
    data = json.load(f)

In [5]:
net = LitModule(shape_edge_index=SHAPE_EMBEDDING_CFG.EDGE_INDEX,
                shape_pose_n_features=SHAPE_EMBEDDING_CFG.POSE_N_FEATURES,
                shape_n_hidden=SHAPE_EMBEDDING_CFG.N_HIDDEN,
                shape_out_features=SHAPE_EMBEDDING_CFG.OUT_FEATURES,
                shape_relation_layers=SHAPE_EMBEDDING_CFG.RELATION_LAYERS,
                class_num=751,
                r50_stride=FT_NET_CFG.R50_STRIDE,
                r50_pretrained_weight=FT_NET_CFG.PRETRAINED)


/Users/jurgendn/miniconda3/envs/deepLearning/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [6]:
img = Image.open("/Users/jurgendn/Downloads/pose-02.jpg")
img = img.resize(size=(256, 128))
img = T.ToTensor()(img)
img = img.unsqueeze(0)

In [7]:
pose = data[0]['pose_landmarks']
pose = torch.FloatTensor(pose)

In [10]:
net.forward(x_image=img, x_pose_features=pose.unsqueeze(0), edge_index=edge_index).shape

torch.Size([1, 1, 1024])

In [9]:
neptune_logger = NeptuneLogger(
    project="jurgen/person-reid",
    api_key=
    "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vbmV3LXVpLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI2MWM3OWRkYi0yMTFlLTQzNjMtOGEzOS0yOGI0MjUxNmRkNjkifQ==",
    log_model_checkpoints=False,
)